In [1]:
import sys
import os
sys.path.insert(0,'../../models/')

import pandas as pd

import sqlite3
from jinja2 import Environment, FileSystemLoader

from utils import open_model
from utils import execute_query
from utils import predict_model

In [2]:
# PATHS
PATH_MODEl = '../../models/trained_models'
PATH_DB = '../../data/DB'
PATH_TEMPLATES = '../../models/templates'

In [3]:
# FILES NAME
FILE_R_FOREST = "r_forest.sav"
FILE_MODELING_DATA = "modeling_data.sql.j2"

In [4]:
# carrega o modelo
r_forest = open_model(PATH_MODEl, FILE_R_FOREST)
r_forest

RandomForestClassifier(random_state=124)

In [5]:
# carrega base de modelagem do DB
modeling_data = execute_query(PATH_DB, PATH_TEMPLATES, FILE_MODELING_DATA, ids='id')
modeling_data.head(3)

,id,OVD_t1,OVD_t2,OVD_sum,pay_normal,new_balance,label,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
0,58987402,0,0,0,1,0.0,0,5,1241.000000,3,62000.0,2,15,9,95,4,60023,1.000000
1,58995151,0,0,0,1,588720.0,1,7,1286.156813,2,126000.0,2,11,9,115,4,450028,224.267697
2,58997200,0,0,0,2,840000.0,0,7,1376.000000,1,150000.0,2,11,-1,110,4,60000,219.248717


In [6]:
case = modeling_data.drop(columns=['id','label']).iloc[1].to_list()

predict_model(case=case, model=r_forest)

array([1])

In [7]:
# inicia o jinja
templateLoader = FileSystemLoader("../../models/templates/")
environment = Environment(loader=templateLoader)

# cria um banco de dados no caminho especificado
conn = sqlite3.connect('../../data/DB/credit_risk')
cursor = conn.cursor()

In [8]:
FILE_VALIDACAO = environment.get_template("validacao_data.sql.j2")
# carrega template do jinja
query_customer = environment.get_template(FILE_VALIDACAO).render()
# exibe a tabela de clientes
validacao = pd.read_sql_query(query_customer, conn)
validacao

,id,OVD_t1,OVD_t2,OVD_sum,pay_normal,new_balance,label,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
0,58982754,1,0,1,35,36091.2,1,5,1322.0,3,195000.0,2,15,5,90,3,60033,240.831892
1,54985177,0,0,0,36,0.0,0,7,1262.0,3,121000.0,2,11,5,106,5,151300,387.356425
2,58990663,0,0,0,1,0.0,0,7,1314.5,3,150000.0,2,11,10,113,4,60023,187.082869
3,58990628,0,0,0,21,24272.4,0,7,1335.5,3,150000.0,2,11,9,105,5,350050,412.310563
4,54983915,0,0,0,3,-4.8,0,5,1256.0,3,110000.0,2,15,9,107,5,60018,1.000000


In [9]:
# fechar conexao
conn.close()

In [10]:
case2 = validacao.drop(columns=['id','label'])#.iloc[0].to_list()

" ".join(str(x) for x in case2)

'OVD_t1 OVD_t2 OVD_sum pay_normal new_balance fea_1 fea_2 fea_3 fea_4 fea_5 fea_6 fea_7 fea_8 fea_9 fea_10 fea_11'

In [11]:
predict_model(case=case2, model=r_forest)

array([1, 0, 0, 0, 0])